In [1]:
import torch
import sys
import torch
import os.path as osp
import tqdm
import sys
import numpy as np
sys.path.append('/afs/cern.ch/work/m/mgarciam/private/mlpf/')
from src.dataset.dataset import SimpleIterDataset
from src.utils.utils import to_filelist
from torch.utils.data import DataLoader
import dgl


/afs/cern.ch/work/m/mgarciam/private/miniconda/miniconda3/envs/graphgps/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
class Args:
    def __init__(self):
        self.data_train = ['/eos/user/m/mgarciam/datasets/pflow/tree2.root']
        self.data_val = ['/eos/user/m/mgarciam/datasets/pflow/tree2.root']
        #self.data_train = files_train
        self.data_config = '/afs/cern.ch/work/m/mgarciam/private/mlpf/config_files/config_2_newlinks.yaml'
        self.extra_selection = None
        self.train_val_split = 0.8
        self.data_fraction = 1
        self.file_fraction = 1
        self.fetch_by_files = False
        self.fetch_step = 0.01
        self.steps_per_epoch = None
        self.in_memory = False
        self.local_rank = None
        self.copy_inputs = False
        self.no_remake_weights = False
        self.batch_size = 10
        self.num_workers = 0
        self.demo = False
        self.laplace = False
        self.diffs = False
        self.class_edges = False
args = Args()
train_range = (0, args.train_val_split)
train_file_dict, train_files = to_filelist(args, 'train')

In [18]:
train_data = SimpleIterDataset(train_file_dict, args.data_config, for_training=True,
                                   extra_selection=args.extra_selection,
                                   remake_weights= True,
                                   load_range_and_fraction=(train_range, args.data_fraction),
                                   file_fraction=args.file_fraction,
                                   fetch_by_files=args.fetch_by_files,
                                   fetch_step=args.fetch_step,
                                   infinity_mode= False,
                                   in_memory=args.in_memory,
                                   async_load = False,
                                   name='train')

In [19]:
iterator = iter(train_data)

=== Restarting DataIter train, seed=None ===


In [33]:

g,gt = next(iterator)

In [35]:
g

Graph(num_nodes=410, num_edges=168100,
      ndata_schemes={'h': Scheme(shape=(7,), dtype=torch.float32), 'pos_hits': Scheme(shape=(3,), dtype=torch.float32), 'pos_hits_norm': Scheme(shape=(3,), dtype=torch.float32), 'hit_type': Scheme(shape=(3,), dtype=torch.int64), 'p_hits': Scheme(shape=(1,), dtype=torch.float32), 'e_hits': Scheme(shape=(1,), dtype=torch.float32), 'particle_number': Scheme(shape=(1,), dtype=torch.float32)}
      edata_schemes={})

In [36]:
g.ndata['particle_number'].unique()

tensor([0.])

In [21]:
gt

tensor([[ -0.0332,   0.0733,   0.9968,   0.5160],
        [-10.0000, -10.0000, -10.0000, -10.0000]])

In [10]:
from src.dataset.functions_graph import graph_batch_func
train_loader = DataLoader(train_data, batch_size=args.batch_size, drop_last=True, pin_memory=True,
                          num_workers=min(args.num_workers, int(len(train_files) * args.file_fraction)),
                          collate_fn = graph_batch_func,
                          persistent_workers=args.num_workers > 0 and args.steps_per_epoch is not None)
iterator = iter(train_loader)

=== Restarting DataIter train, seed=None ===


In [11]:
g,y = next(iterator)

In [14]:
y.shape

torch.Size([20, 4])

In [62]:
from src.models.wrapper.example_gravnet_model import GraphTransformerNetWrapper
newmodel = GraphTransformerNetWrapper('cpu')
newmodel.load_state_dict(torch.load('/afs/cern.ch/work/m/mgarciam/private/mlpf/models_trained/_epoch-97_state.pt', map_location='cpu'))


<All keys matched successfully>

In [63]:
pred = newmodel(g)

In [67]:

_, S = pred.shape
xj = torch.nn.functional.normalize(pred[:, 0:3], dim=1)
bj = torch.sigmoid(torch.reshape(pred[:, 3], [-1, 1]))
distance_threshold = torch.sigmoid(torch.reshape(pred[:, 4:7], [-1, 3]))
energy_correction = torch.sigmoid(torch.reshape(pred[:, 7], [-1, 1]))
dev = g.device
print(xj.shape, bj.shape, distance_threshold.shape, energy_correction.shape)
clustering_index_l = g.ndata['particle_number']

len_batch = len(g.batch_num_nodes())
batch_numbers = torch.repeat_interleave(
    torch.range(0, len_batch - 1), g.batch_num_nodes()
)

torch.Size([187, 3]) torch.Size([187, 1]) torch.Size([187, 3]) torch.Size([187, 1])


/tmp/mgarciam/ipykernel_27928/2716966644.py:12: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  torch.range(0, len_batch - 1), g.batch_num_nodes()


In [68]:
beta = bj.view(-1)
cluster_index_per_event = clustering_index_l.view(-1).long()
batch = batch_numbers.long()
cluster_space_coords = xj
qmin = 0.1
s_B = 1.
noise_cluster_index = 0
beta_stabilizing = 'soft_q_scaling'
huberize_norm_for_V_attractive = False
beta_term_option = 'paper'
return_components = False


In [11]:
from torch_scatter import scatter_max, scatter_add, scatter_mean
from src.layers.object_cond import batch_cluster_indices, scatter_count, scatter_counts_to_indices
cluster_index, n_clusters_per_event = batch_cluster_indices(cluster_index_per_event, batch)
n_clusters = n_clusters_per_event.sum()
n_hits, cluster_space_dim = cluster_space_coords.size()
batch_size = batch.max()+1
n_hits_per_event = scatter_count(batch)

# Index of cluster -> event (n_clusters,)
batch_cluster = scatter_counts_to_indices(n_clusters_per_event)

# Per-hit boolean, indicating whether hit is sig or noise
is_noise = cluster_index_per_event == noise_cluster_index
is_sig = ~is_noise
n_hits_sig = is_sig.sum()
n_sig_hits_per_event = scatter_count(batch[is_sig])

In [69]:
# Per-cluster boolean, indicating whether cluster is an object or noise
is_object = scatter_max(is_sig.long(), cluster_index)[0].bool()
is_noise_cluster = ~is_object

# FIXME: This assumes noise_cluster_index == 0!!
# Not sure how to do this in a performant way in case noise_cluster_index != 0
if noise_cluster_index != 0: raise NotImplementedError
object_index_per_event = cluster_index_per_event[is_sig] - 1
object_index, n_objects_per_event = batch_cluster_indices(object_index_per_event, batch[is_sig])
n_hits_per_object = scatter_count(object_index)
batch_object = batch_cluster[is_object]
n_objects = is_object.sum()

assert object_index.size() == (n_hits_sig,)
assert is_object.size() == (n_clusters,)
assert torch.all(n_hits_per_object > 0)
assert object_index.max()+1 == n_objects


In [70]:
if beta_stabilizing == 'paper':
    q = beta.arctanh()**2 + qmin
elif beta_stabilizing == 'clip':
    beta = beta.clip(0., 1-1e-4)
    q = beta.arctanh()**2 + qmin
elif beta_stabilizing == 'soft_q_scaling':
    q = (beta.clip(0., 1-1e-4)/1.002).arctanh()**2 + qmin
else:
    raise ValueError(f'beta_stablizing mode {beta_stabilizing} is not known')
assert q.device == dev
assert q.size() == (n_hits,)

# Calculate q_alpha, the max q per object, and the indices of said maxima
q_alpha, index_alpha = scatter_max(q[is_sig], object_index)
assert q_alpha.size() == (n_objects,)

# Get the cluster space coordinates and betas for these maxima hits too
x_alpha = cluster_space_coords[is_sig][index_alpha]
beta_alpha = beta[is_sig][index_alpha]
assert x_alpha.size() == (n_objects, cluster_space_dim)
assert beta_alpha.size() == (n_objects,)


In [72]:
g.ndata['hit_type'][index_alpha]

tensor([[1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0],
        [1, 0, 0]])

In [78]:
len([2,2])

2

In [35]:
positions_particles_pred = g.ndata['pos_hits_norm'][is_sig][index_alpha]
positions_particles_pred = positions_particles_pred+distance_threshold[is_sig][index_alpha]

e_particles_pred = g.ndata['e_hits'][is_sig][index_alpha]
e_particles_pred = e_particles_pred*energy_correction[is_sig][index_alpha]

In [42]:
x_particles = y[:,0:3]
e_particles = y[:,3].unsqueeze(1)

In [51]:
loss_E = torch.mean(torch.square((e_particles_pred - e_particles)/e_particles))

In [49]:
loss_E = torch.mean(torch.square((e_particles_pred - e_particles)/e_particles))
loss_mse = torch.nn.MSELoss()
loss_x = loss_mse(positions_particles_pred,x_particles)